In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time


In [2]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0
X

array([[[[0.71764706],
         [0.70588235],
         [0.77254902],
         ...,
         [0.61960784],
         [0.69411765],
         [0.7372549 ]],

        [[0.76078431],
         [0.67843137],
         [0.70980392],
         ...,
         [0.63137255],
         [0.68235294],
         [0.69411765]],

        [[0.75294118],
         [0.70196078],
         [0.81960784],
         ...,
         [0.63137255],
         [0.6627451 ],
         [0.68627451]],

        ...,

        [[0.94509804],
         [0.9372549 ],
         [0.9254902 ],
         ...,
         [0.83529412],
         [0.80784314],
         [0.36078431]],

        [[0.9372549 ],
         [0.92156863],
         [0.92156863],
         ...,
         [0.76078431],
         [0.72941176],
         [0.39607843]],

        [[0.91764706],
         [0.90196078],
         [0.90196078],
         ...,
         [0.70588235],
         [0.6745098 ],
         [0.40784314]]],


       [[[0.51372549],
         [0.52941176],
         [0.53

In [10]:
'''
Keras
'''
NAME = "Cats-vs-dogs-CNN-{}".format(int(time.time()))

model = Sequential()

model.add(Conv2D(64, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

tensorboard = TensorBoard(log_dir="/content/clouderizer/catdogs/out/{}".format(NAME))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=100, epochs=20, validation_split=0.1, callbacks=[tensorboard])

Train on 22500 samples, validate on 2500 samples
Epoch 1/20
22500/22500 [==============================] - 22s 978us/step - loss: 0.6896 - acc: 0.5802 - val_loss: 0.6470 - val_acc: 0.6312
Epoch 2/20
22500/22500 [==============================] - 22s 961us/step - loss: 0.6389 - acc: 0.6400 - val_loss: 0.6065 - val_acc: 0.7060
Epoch 3/20
22500/22500 [==============================] - 22s 963us/step - loss: 0.5791 - acc: 0.6965 - val_loss: 0.5474 - val_acc: 0.7228
Epoch 4/20
22500/22500 [==============================] - 21s 953us/step - loss: 0.5140 - acc: 0.7461 - val_loss: 0.4905 - val_acc: 0.7692
Epoch 5/20
22500/22500 [==============================] - 21s 938us/step - loss: 0.4618 - acc: 0.7817 - val_loss: 0.4616 - val_acc: 0.7844
Epoch 6/20
22500/22500 [==============================] - 21s 942us/step - loss: 0.4063 - acc: 0.8142 - val_loss: 0.4588 - val_acc: 0.7932
Epoch 7/20
22500/22500 [==============================] - 21s 936us/step - loss: 0.3695 - acc: 0.8392 - val_loss: 0.4

In [ ]:
'''
Optimization with tensorboard
Tries 3 different Conv block, 3 different filter_depth and 3 different fully connected dense layer.
Tries all these 3*3*3=27 conditions. Puts each output in out folder.
'''
dense_layers = [2,1,0]
filters = [32, 64, 128]
conv_blocks = [1, 2, 3]
for dense_layer in dense_layers:
    for filter_depth in filters:
        for conv_block in conv_blocks:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_block, filter_depth, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(filter_depth, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_block-1):
                model.add(Conv2D(filter_depth, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(500))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="/content/clouderizer/catdogs/out/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=10,
                      validation_split=0.3,
                      callbacks=[tensorboard])

1-conv-32-nodes-2-dense-1536060447
Train on 17500 samples, validate on 7500 samples
Epoch 1/10
17500/17500 [==============================] - 41s 2ms/step - loss: 0.6187 - acc: 0.6624 - val_loss: 0.5577 - val_acc: 0.7265
Epoch 2/10
17500/17500 [==============================] - 39s 2ms/step - loss: 0.4564 - acc: 0.7839 - val_loss: 0.5518 - val_acc: 0.7281
Epoch 3/10
17500/17500 [==============================] - 40s 2ms/step - loss: 0.2923 - acc: 0.8723 - val_loss: 0.7204 - val_acc: 0.7161
Epoch 4/10
17500/17500 [==============================] - 39s 2ms/step - loss: 0.1212 - acc: 0.9531 - val_loss: 1.0110 - val_acc: 0.7065
Epoch 5/10
17500/17500 [==============================] - 39s 2ms/step - loss: 0.0697 - acc: 0.9770 - val_loss: 1.2330 - val_acc: 0.7067
Epoch 6/10
17500/17500 [==============================] - 40s 2ms/step - loss: 0.0355 - acc: 0.9881 - val_loss: 1.4997 - val_acc: 0.7137
Epoch 7/10
17500/17500 [==============================] - 39s 2ms/step - loss: 0.0200 - acc: 0